In [1]:
#melakukan import library yang dibutuhkan
import pyspark
import numpy as np
import pandas as pd

In [2]:
#memulai Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("Proyek_Sentiment Analysis") \
    .getOrCreate()

In [3]:
spark

### Load Dataset

In [4]:
# melakukan load dataset menggunakan library pandas (dikarenakan Spark load data tidak sesuai)
df = pd.read_csv("D:/semester8/PDB/Proyek/IMDB Dataset.csv")

In [5]:
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from collections import Counter
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [7]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [12]:
# melakukan load dataset kedalam Spark dan menampilkannya
reviews = spark.createDataFrame(df)
reviews.show(10)

+--------------------+---------+
|              review|sentiment|
+--------------------+---------+
|One of the other ...| positive|
|A wonderful littl...| positive|
|I thought this wa...| positive|
|Basically there's...| negative|
|Petter Mattei's "...| positive|
|Probably my all-t...| positive|
|I sure would like...| positive|
|This show was an ...| negative|
|Encouraged by the...| negative|
|If you like origi...| positive|
+--------------------+---------+
only showing top 10 rows



In [13]:
reviews.printSchema()

root
 |-- review: string (nullable = true)
 |-- sentiment: string (nullable = true)



#### Melakukan Konversi Data ke format Parquet

In [14]:
reviews.write.format("parquet").mode("overwrite").save("D:/semester8/PDB/Proyek/Data_Parquet")

### Load Data Parquet

In [17]:
data_parq=spark.read.format("parquet").load("D:/semester8/PDB/Proyek/Data_Parquet")

In [18]:
data_parq.show(10)

+--------------------+---------+
|              review|sentiment|
+--------------------+---------+
|Jess Franco makes...| negative|
|Really enjoyed th...| positive|
|I read several mi...| positive|
|Absolutely one of...| positive|
|True stories make...| positive|
|I saw The Big Bad...| positive|
|Actually I'm stil...| negative|
|Let me say first ...| positive|
|I write this revi...| positive|
|The odd mixture o...| negative|
+--------------------+---------+
only showing top 10 rows



In [19]:
data_parq.printSchema()

root
 |-- review: string (nullable = true)
 |-- sentiment: string (nullable = true)



In [20]:
# melihat deskripsi dari data yang digunakan
data_parq.describe().show()

+-------+--------------------+---------+
|summary|              review|sentiment|
+-------+--------------------+---------+
|  count|               50000|    50000|
|   mean|                null|     null|
| stddev|                null|     null|
|    min|A Turkish Bat...| negative|
|    max|ý thýnk uzak ýs t...| positive|
+-------+--------------------+---------+



In [21]:
# melakukan pengecekan dan menghapus nilai null 
from pyspark.sql.functions import count

def my_count(df_in):
    df_in.agg( *[ count(c).alias(c) for c in df_in.columns ] ).show()

In [22]:
my_count(data_parq)

+------+---------+
|review|sentiment|
+------+---------+
| 50000|    50000|
+------+---------+

